In [41]:
%matplotlib widget

import h5py
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

def get_xas(scanNumber):
    global projectPath
    global baseAtom
    if scanNumber < 10:
        filename = baseAtom+'_'+'000'+str(scanNumber)
    elif scanNumber < 100:
        filename = baseAtom+'_'+'00'+str(scanNumber)
    elif scanNumber < 1000:    
        filename = baseAtom+'_'+'0'+str(scanNumber)
    else:    
        filename = baseAtom+'_'+str(scanNumber)
    data = np.loadtxt(projectPath+'\\XAS\\'+filename+'.xas', comments='#')
    photonEnergy = data[:,0]
    tey = data[:,1]
    tfy = data[:,2]
    rmu = data[:,3]
    return photonEnergy,tey,tfy,rmu

def elastic_shift(pixelData):
    global energyDispersion
    peaks, _ = signal.find_peaks(pixelData,height=(np.max(pixelData)/10 if np.max(pixelData)/10>10 else 10),width=3)
    xdataPixel = np.arange(len(pixelData))
    xdataPixel = xdataPixel[(peaks[-1]-2000):(peaks[-1]+200)]
    energyData = pixelData[(peaks[-1]-2000):(peaks[-1]+200)]
    xDataEnergy = (xdataPixel - peaks[-1]) * energyDispersion * -1
    return xDataEnergy,energyData

def x_corr(refData, uncorrData):
    corr = signal.correlate(refData, uncorrData)
    lag = np.argmax(corr)
    corrData = np.roll(uncorrData, lag)
    return corrData

def get_rixs(scannumber):
    global projectPath
    global baseAtom
    if scannumber < 10:
        filename = baseAtom+'_'+'000'+str(scannumber)
    elif scannumber < 100:
        filename = baseAtom+'_'+'00'+str(scannumber)
    elif scannumber < 1000:    
        filename = baseAtom+'_'+'0'+str(scannumber)
    else:    
        filename = baseAtom+'_'+str(scannumber)
    f1 = h5py.File(projectPath+'\\RIXS\\'+filename+'_d1.h5', 'r')
    f2 = h5py.File(projectPath+'\\RIXS\\'+filename+'_d2.h5', 'r')
    f3 = h5py.File(projectPath+'\\RIXS\\'+filename+'_d3.h5', 'r')
    ccd1 = np.array(f1['entry']['analysis']['spectrum'][()])
    ccd2 = np.array(f2['entry']['analysis']['spectrum'][()])
    ccd3 = np.array(f3['entry']['analysis']['spectrum'][()])
    ccd1 = x_corr(ccd2,ccd1)
    ccd3 = x_corr(ccd2,ccd3)
    xdata,tempData = elastic_shift(ccd1+ccd2+ccd3)
    return xdata,tempData

def get_scan(scans):
    for i,scannumber in enumerate(scans):
        if i == 0:
            xdata,ydata = get_rixs(scannumber)
            refdata = ydata
            sumdata = ydata
        else:
            _,ydata = get_rixs(scannumber)
            ydata = x_corr(refdata,ydata)
            sumdata = sumdata+ydata
    return xdata,sumdata

In [12]:
def plot_xas(scans,dataType):
    for x in scans:
        E,TEY,TFY,RMU = get_xas(x)
        if dataType == 'TEY':
            plt.plot(E,TEY,label='TEY : '+str(x))
        else:
            plt.plot(E,TFY,label='TFY : '+str(x))
        plt.xlabel('Photon Energy ( eV )')
        plt.ylabel('Intensity ( arb. unit )')

def plot_rixs(scans):
    X,Y = get_scan(scans)
    plt.plot(X,Y,label = 'RIXS : '+str(scans[0])+'-'+str(scans[-1]))
    plt.xlabel('Energy Loss ( eV )')
    plt.ylabel('Intensity ( arb. unit )')

def do_plot(fileType,start,end):
    scans = lambda start,end : np.arange(start,end+1,1)
    if fileType == 'TEY':
        plot_xas(scans(start,end),'TEY')
    elif fileType == 'TFY':
        plot_xas(scans(start,end),'TFY')
    else:
        plot_rixs(scans(start,end))
    plt.legend()

In [3]:
import ipywidgets as widgets
import ipympl
from IPython.display import clear_output
import os

In [15]:
path = widgets.Text(
    value=r'X:\\RIXS\\Asmara',
    placeholder='Parent Path',
    description='Project Path:',
    disabled=False
)
display(path)

Text(value='X:\\\\RIXS\\\\Asmara', description='Project Path:', placeholder='Parent Path')

In [16]:
base = widgets.Text(
    value='O',
    placeholder='Element',
    description='Prefix:',
    disabled=False
)
display(base)

Text(value='O', description='Prefix:', placeholder='Element')

In [17]:
dispersion = widgets.FloatText(
    value='1',
    placeholder='eV/subpixel',
    description='Dispersion:',
    disabled=False
)
display(dispersion)

FloatText(value=1.0, description='Dispersion:')

In [18]:
file = widgets.Dropdown(
    options=['RIXS','TEY','TFY'],
    value='RIXS',
    description='Data Type:',
    disabled=False,
)
display(file)

Dropdown(description='Data Type:', options=('RIXS', 'TEY', 'TFY'), value='RIXS')

In [42]:
scan = widgets.IntRangeSlider(
    value=[10, 20],
    min=0,
    max=30,
    step=1,
    description='Scan:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)
display(scan)

def on_value_change(change):
    scan.min = (np.round(scan.value[0] * 2 - scan.value[1]  - 10 ) if np.round(scan.value[0] * 2 - scan.value[1] - 10 ) > 0 else 0)
    scan.max = (np.round(scan.value[1] * 2 - scan.value[0]  + 10 ) if np.round(scan.value[1] * 2 - scan.value[0] + 10 ) < 9999 else 9999)

scan.observe(on_value_change, names='value')

IntRangeSlider(value=(10, 100), continuous_update=False, description='Scan:')

In [9]:
compare = widgets.Checkbox(
    value=False,
    description='Compare',
    disabled=False,
    indent=False
)
display(compare)

Checkbox(value=False, description='Compare', indent=False)

In [ ]:
plot = widgets.Button(description="Plot")
output = widgets.Output()
display(plot,output)
plt.figure()

def on_button_clicked(b):
#     with output:
    global projectPath,baseAtom,energyDispersion
    projectPath = path.value
    baseAtom = base.value
    energyDispersion = dispersion.value
    fileType = file.value
    start = scan.value[0]
    end = scan.value[1]
    compareType = compare.value
    if compareType == True:
        pass
    else:
        plt.clf()
    do_plot(fileType,start,end)


plot.on_click(on_button_clicked)